## Set up network and install software

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py


# Import topology_variables

# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'

path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

import write_value


In [ ]:
try:
    slice = fablib.get_slice(name=selected_slice)
    server = slice.get_node(name = node1_name)
    client = slice.get_node(name = node2_name)
    
    # Install apache on server node
    apache_root_dir = "/var/www/html" # Default Root Directory to Store Web Server Files
    print("Installing Apache...")
    stdout, stderr = server.execute("sudo apt-get update && sudo apt-get install -y apache2")
    print("Finished installing Apache.")
    
    # Set up firewall
    commands = [
        "sudo ufw allow 'Apache'",                                # Allow Port 80 Through Firewall
        "sudo ufw allow ssh",                                     # Allow Port 22 Through Firewall (IMPORTANT)
        "echo 'y' | sudo ufw enable"                              # Enable the Firewall
    ]
    print("Setting up firewall...")
    for command in commands:
        stdout, stderr = server.execute(command)
    print("Finished setting up firewall.")
    
    # Check firewall is active
    stdout, stderr = server.execute('sudo ufw status')
    print(stdout)
    
    #Ensure apache service is active
    stdout, stderr = server.execute('sudo systemctl status apache2')
    print(stdout)
    
    #Change dir where Apache files are stored
    apache_root_dir = "/home/ubuntu/apache_contents"
    change_root = f"mkdir -p {apache_root_dir} && sudo sed -i 's,/var/www/html,{apache_root_dir},g' /etc/apache2/sites-available/000-default.conf && sudo sed -i 's,/var/www/html,{apache_root_dir},g' /etc/apache2/sites-available/default-ssl.conf && sudo sed -i 's,/var/www/,{apache_root_dir},g' /etc/apache2/apache2.conf"
    stdout, stderr = server.execute(change_root)
    print(stderr)
    
    stdout, stderr = server.execute("sudo service apache2 restart")
    print(stderr)

    success = True
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to error.')